# 1. 라이브러리 불러오기 

In [1]:
# 데이터 처리 라이브러리
import os
import os.path as pth
from tqdm import tqdm
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt 
from sklearn.model_selection import train_test_split

# Tensorflow 관련 라이브러리
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import models, layers
from tensorflow.keras.models import Sequential,Model, load_model
from tensorflow.keras.layers import Conv2D, Dense, Dropout, BatchNormalization, Flatten, Add, Activation, Dense, Input
from tensorflow.keras.layers import MaxPooling2D, AveragePooling2D, GlobalAveragePooling2D
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.optimizers import Adam, Nadam

# GPU 설정
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    # 특정 GPU에 1GB 메모리만 할당하도록 제한
    try:
        tf.config.experimental.set_visible_devices(gpus[1], 'GPU')
        tf.config.experimental.set_virtual_device_configuration(
            gpus[1],
            [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=7250)])
    except RuntimeError as e:
    # 프로그램 시작시에 가상 장치가 설정되어야만 합니다
        print(e)

# # mirrored_strategy = tf.distribute.MirroredStrategy()
# mirrored_strategy = tf.distribute.MirroredStrategy(devices=["/job:localhost/replica:0/task:0/device:GPU:0", "/job:localhost/replica:0/task:0/device:GPU:1"],
# cross_device_ops = tf.distribute.HierarchicalCopyAllReduce())
print(gpus)

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU'), PhysicalDevice(name='/physical_device:GPU:1', device_type='GPU')]


In [2]:
strategy = tf.distribute.MirroredStrategy()
tf.config.set_soft_device_placement(True)

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0', '/job:localhost/replica:0/task:0/device:GPU:1')


# 2. 데이터 불러오기 

## 2-1. Labeling 데이터프레임 불러오기 

In [2]:
# 학습 데이터 준비 
path = "./data/landmark"  
label_df = pd.read_csv(path + '/category.csv') # 각 랜드마크별 label
label_dict = dict(label_df[['landmark_name', 'landmark_id']].values)
label_dict_reverse = dict(label_df.values)

## 2-2. Train Data 경로 불러오기

In [3]:
## Train 파일(JPG)명과 label 정보를 담은 데이터 프레임 생성
train_dirs = path + '/train'
files = []
categories=[]
for img_dir in os.listdir(train_dirs):
    img_dir_list = train_dirs + '/' + img_dir
    
    for filename in os.listdir(img_dir_list):
        file_dir = img_dir + '/' + filename
        files.append(file_dir)
        categories.append(label_dict[img_dir])
            
train_data=pd.DataFrame(
                    {"file":files,
                    "label":categories}
                )    

train_data

,file,label
0,수영구청/수영구청_011.JPG,794
1,수영구청/수영구청_008.JPG,794
2,수영구청/수영구청_060.JPG,794
3,수영구청/수영구청_075.JPG,794
4,수영구청/수영구청_023.JPG,794
...,...,...
88097,수영구 생활문화센터/수영구_생활문화센터_118.JPG,310
88098,수영구 생활문화센터/수영구_생활문화센터_038.JPG,310
88099,수영구 생활문화센터/수영구_생활문화센터_010.JPG,310
88100,수영구 생활문화센터/수영구_생활문화센터_050.JPG,310


## 2-3. Test Data 경로 불러오기

In [4]:
# test 데이터 안 '6' 폴더에 있는 체크포인트 오류 데이터 제거 후 실행
test_dirs = path + '/test'
files = []
ids=[]
for img_cat in os.listdir(test_dirs):
    id_dir = test_dirs + '/' + img_cat
    for filename in os.listdir(id_dir):
        files.append(id_dir + '/' +filename)
        ids.append(filename.split('.JPG')[0])
                           
test_data = pd.DataFrame(
                    {"file":files,
                    "id":ids}
                )    

test_data

,file,id
0,./data/landmark/test/0/0hmnf5orki.JPG,0hmnf5orki
1,./data/landmark/test/0/0bgj9co0zl.JPG,0bgj9co0zl
2,./data/landmark/test/0/03123sl42g.JPG,03123sl42g
3,./data/landmark/test/0/0vwaki2su2.JPG,0vwaki2su2
4,./data/landmark/test/0/09jgq862fk.JPG,09jgq862fk
...,...,...
37959,./data/landmark/test/h/hf700dgjt3.JPG,hf700dgjt3
37960,./data/landmark/test/h/hf5vrn6vdx.JPG,hf5vrn6vdx
37961,./data/landmark/test/h/hr5xdtptl2.JPG,hr5xdtptl2
37962,./data/landmark/test/h/hj455fxmjr.JPG,hj455fxmjr


# 3. TFRecord 데이터 처리 

## 3-2. TFrecord 파일 TRAIN, VALIDATION, TEST(불러오기)

In [5]:
path = "./data/landmark" 
train_tfrecord_path = pth.join(path, 'tf_record_train.tfrecords')
valid_tfrecord_path = pth.join(path, 'tf_record_valid.tfrecords')

BUFFER_SIZE = 128
BATCH_SIZE = 10
NUM_CLASS = 1049

In [6]:
# with strategy.scope():
with tf.device('/device:GPU:1'):
    image_feature_description = {
        'image_raw': tf.io.FixedLenFeature([], tf.string),
        'randmark_id': tf.io.FixedLenFeature([], tf.int64),
        # 'id': tf.io.FixedLenFeature([], tf.string),
    }
    
def _parse_image_function(example_proto):
    return tf.io.parse_single_example(example_proto, image_feature_description)

def map_func(target_record):
        img = target_record['image_raw']
        label = target_record['randmark_id']
        img = tf.image.decode_jpeg(img, channels=3)
        if tf.random.uniform([]) > 0.5:
            img = tf.image.flip_left_right(img)
        img = tf.cast(img, tf.float32)
        img = tf.dtypes.cast(img, tf.float32)
        return img, label

def map_func_valid(target_record):
    img = target_record['image_raw']
    label = target_record['randmark_id']
    img = tf.image.decode_jpeg(img, channels=3)
    img = tf.dtypes.cast(img, tf.float32)
    return img, label

def prep_func(image, label):
    result_image = image / 255
    result_image = tf.image.resize(result_image, (270,480))
    onehot_label = tf.one_hot(label, depth=NUM_CLASS)
    return result_image, onehot_label

# with strategy.scope():
with tf.device('/device:GPU:1'):
    dataset = tf.data.TFRecordDataset(train_tfrecord_path, compression_type='GZIP')
    dataset = dataset.map(_parse_image_function, num_parallel_calls=tf.data.experimental.AUTOTUNE)
    dataset = dataset.map(map_func, num_parallel_calls=tf.data.experimental.AUTOTUNE)
    dataset = dataset.shuffle(BUFFER_SIZE)
    dataset = dataset.batch(BATCH_SIZE)
    dataset = dataset.map(prep_func, num_parallel_calls=tf.data.experimental.AUTOTUNE)
    dataset = dataset.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)

    valid_dataset = tf.data.TFRecordDataset(valid_tfrecord_path, compression_type='GZIP')
    valid_dataset = valid_dataset.map(_parse_image_function, num_parallel_calls=tf.data.experimental.AUTOTUNE)
    valid_dataset = valid_dataset.map(map_func_valid, num_parallel_calls=tf.data.experimental.AUTOTUNE)
    valid_dataset = valid_dataset.batch(BATCH_SIZE)
    valid_dataset = valid_dataset.map(prep_func, num_parallel_calls=tf.data.experimental.AUTOTUNE)
    valid_dataset = valid_dataset.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)

In [7]:
test_tfrecord_path = pth.join(path, 'tf_record_test.tfrecords')

# with strategy.scope():
with tf.device('/device:GPU:1'):
    test_image_feature_description = {
        'image_raw': tf.io.FixedLenFeature([], tf.string),
        'id': tf.io.FixedLenFeature([], tf.string),
    }
    
def test_image_function(example_proto):
    return tf.io.parse_single_example(example_proto, test_image_feature_description)

def test_map_func(target_record):
    img = target_record['image_raw']
    label = target_record['id']
    img = tf.image.decode_jpeg(img, channels=3)
    img = tf.dtypes.cast(img, tf.float32)
    return img, label

def test_prep_func(image, label):
    result_image = image / 255.
    result_image = tf.image.resize(result_image, (270,480))    
    return result_image, label
    
# with strategy.scope():
with tf.device('/device:GPU:1'):    
    test_dataset = tf.data.TFRecordDataset(test_tfrecord_path, compression_type='GZIP')
    test_dataset = test_dataset.map(test_image_function, num_parallel_calls=tf.data.experimental.AUTOTUNE)
    test_dataset = test_dataset.map(test_map_func, num_parallel_calls=tf.data.experimental.AUTOTUNE)
    test_dataset = test_dataset.shuffle(BUFFER_SIZE)
    test_dataset = test_dataset.batch(BATCH_SIZE)
    test_dataset = test_dataset.map(test_prep_func, num_parallel_calls=tf.data.experimental.AUTOTUNE)
    test_dataset = test_dataset.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)
    

    

## npy 파일 형식 불러오기

In [ ]:
with tf.device('/device:GPU:1'):  
    x_train = np.load(path + '/x_train.npy')
    x_valid = np.load(path + '/x_valid.npy')
    y_train = np.load(path + '/y_train.npy')
    y_valid = np.load(path + '/y_valid.npy')
#     test_data = np.load(path + "/test_data.npy")

    print(x_train.shape)
    print(y_train.shape)
    print(x_valid.shape)
    print(y_valid.shape)

# 4. 모델링 

## EfficientNet

In [ ]:
from tensorflow.keras import models
from tensorflow.keras.applications import EfficientNetB5 

efficientnet = EfficientNetB5(weights ='imagenet', include_top = False, 
                              input_shape = (224, 224, 3), classifier_activation='softmax')

earlystop = EarlyStopping(patience=5)
learning_rate_reduction=ReduceLROnPlateau(
                        monitor= "val_loss", 
                        patience = 2, 
                        factor = 0.5, 
                        min_lr=1e-7,
                        verbose=1)

model_check = ModelCheckpoint( #에포크마다 현재 가중치를 저장    
        filepath="./landmark_efficientnetb5.h5", #모델 파일 경로
        monitor='val_loss',  # val_loss 가 좋아지지 않으면 모델 파일을 덮어쓰지 않음.
        save_best_only=True)

print(efficientnet.summary())    
# print(#################################################################################)   

callbacks = [earlystop, learning_rate_reduction, model_check]

for layer in efficientnet.layers:
    layer.trainable = True
 
with tf.device('/device:GPU:1'):  
    model = models.Sequential()
    model.add(efficientnet)
    model.add(layers.Flatten())
    model.add(Dense(512, activation='relu', kernel_initializer='he_normal'))
    model.add(BatchNormalization())
    model.add(Dropout(0.5))
    model.add(Dense(1049, activation='softmax')) 
    model.summary()

    model.compile(loss='categorical_crossentropy',
                  optimizer=Adam(learning_rate=0.001),
                  metrics=['accuracy'])
    
    history = model.fit(dataset,

                    epochs=100,
                    validation_data=valid_dataset,
                    callbacks = callbacks)

## DenseNet

In [ ]:
from tensorflow.keras import models
from tensorflow.keras.applications import DenseNet121 

densenet = DenseNet121(weights ='imagenet', include_top = False, 
                       input_shape = (224, 224, 3), pooling='avg')

earlystop = EarlyStopping(patience=5)
learning_rate_reduction=ReduceLROnPlateau(
                        monitor= "val_loss", 
                        patience = 2, 
                        factor = 0.5, 
                        min_lr=1e-7,
                        verbose=1)

model_check = ModelCheckpoint( #에포크마다 현재 가중치를 저장    
        filepath="./landmark_densenet121_avg_1109.h5", #모델 파일 경로
        monitor='val_loss',  # val_loss 가 좋아지지 않으면 모델 파일을 덮어쓰지 않음.
        save_best_only=True)

# print(efficientnet.summary())    
# print(#################################################################################)   

callbacks = [earlystop, learning_rate_reduction, model_check]

for layer in densenet.layers:
    layer.trainable = True
 
with tf.device('/device:GPU:1'):  
    model = models.Sequential()
    model.add(densenet)
    model.add(layers.Flatten())
    model.add(Dense(512, activation=PReLu(), kernel_initializer='he_normal'))
    model.add(BatchNormalization())
    model.add(Dropout(0.5))
    model.add(Dense(1049, activation='softmax')) 
    model.summary()

    model.compile(loss='categorical_crossentropy',
                  optimizer=Adam(learning_rate=0.0001),
                  metrics=['categorical_accuracy'])
    
    history = model.fit(dataset,
                        epochs=20,
                        validation_data=valid_dataset,
                        callbacks = callbacks)

## NASNetMobile

In [ ]:
from tensorflow.keras import models
from tensorflow.keras.applications import NASNetMobile 

nasnetmobile = NASNetMobile(weights = 'imagenet', include_top = False)

earlystop = EarlyStopping(patience=5)
learning_rate_reduction=ReduceLROnPlateau(
                        monitor= "val_loss", 
                        patience = 2, 
                        factor = 0.3, 
                        min_lr=1e-7,
                        verbose=1)

model_check = ModelCheckpoint( #에포크마다 현재 가중치를 저장    
        filepath="./landmark_nasnetmobile_1113.h5", #모델 파일 경로
        monitor='val_loss',  # val_loss 가 좋아지지 않으면 모델 파일을 덮어쓰지 않음.
        save_best_only=True)

# print(efficientnet.summary())    
# print(#################################################################################)   

callbacks = [earlystop, learning_rate_reduction, model_check]

for layer in nasnetmobile.layers:
    layer.trainable = True
 
with strategy.scope(): 
# with tf.device('/device:GPU:1'):  
    model = models.Sequential()
    model.add(nasnetmobile)
    model.add(GlobalAveragePooling2D())
    model.add(Dense(1049, activation='softmax')) 
    model.summary()

    model.compile(loss='categorical_crossentropy',
                  optimizer=Adam(learning_rate=0.0001),
                  metrics=['categorical_accuracy'])
    
    history = model.fit(dataset,
                        epochs=10,
                        validation_data=valid_dataset,
                        callbacks = callbacks)

## DenseNet201

#### 11/12 멀티 gpu 사용 전 metrics에서의 오류 해결을 위해 모델을 우선 생성

In [10]:
from tensorflow.keras import models
from tensorflow.keras.applications import DenseNet201 

densenet201 = DenseNet201(weights ='imagenet', include_top = False, input_shape=(270,480,3))

earlystop = EarlyStopping(patience=5)
learning_rate_reduction=ReduceLROnPlateau(
                        monitor= "val_loss", 
                        patience = 1, 
                        factor = 0.7, 
                        min_lr=1e-7,
                        verbose=1)

model_check = ModelCheckpoint( #에포크마다 현재 가중치를 저장    
        filepath="./landmark_densenet201_1113.h5", #모델 파일 경로
        monitor='val_loss',  # val_loss 가 좋아지지 않으면 모델 파일을 덮어쓰지 않음.
        save_best_only=True)

# print(#################################################################################)   

callbacks = [earlystop, learning_rate_reduction, model_check]

for layer in densenet201.layers:
    layer.trainable = True
 
# with strategy.scope(): 
with tf.device('/device:GPU:1'):  
    model = models.Sequential()
    model.add(densenet201)
    model.add(GlobalAveragePooling2D())
    model.add(Dense(1049, activation='softmax')) 
    model.summary()

    model.compile(loss='categorical_crossentropy',
                  optimizer=Adam(learning_rate=0.0001),
                  metrics=['accuracy'])
    
    history = model.fit(dataset,
                        epochs=10,
                        validation_data=valid_dataset,
                        callbacks = callbacks)

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
densenet201 (Functional)     (None, 8, 15, 1920)       18321984  
_________________________________________________________________
global_average_pooling2d (Gl (None, 1920)              0         
_________________________________________________________________
dense (Dense)                (None, 1049)              2015129   
Total params: 20,337,113
Trainable params: 20,108,057
Non-trainable params: 229,056
_________________________________________________________________
Epoch 1/10
8811/8811 [==============================] - 5185s 588ms/step - loss: 2.1972 - accuracy: 0.7080 - val_loss: 0.2124 - val_accuracy: 0.9543
Epoch 2/10
  86/8811 [..............................] - ETA: 1:19:23 - loss: 0.3148 - accuracy: 0.9491

KeyboardInterrupt: 

## Load하여 멀티 GPU로 재학습

In [8]:
from tensorflow.keras.models import load_model
# with strategy.scope():
with tf.device('/device:GPU:1'):  
    densenet201_load = load_model("landmark_densenet201_1113_update.h5")
    densenet201_load.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
densenet201 (Functional)     (None, 8, 15, 1920)       18321984  
_________________________________________________________________
global_average_pooling2d (Gl (None, 1920)              0         
_________________________________________________________________
dense (Dense)                (None, 1049)              2015129   
Total params: 20,337,113
Trainable params: 20,108,057
Non-trainable params: 229,056
_________________________________________________________________


In [ ]:
earlystop = EarlyStopping(patience=5)
learning_rate_reduction=ReduceLROnPlateau(
                        monitor= "val_loss", 
                        patience = 1, 
                        factor = 0.8, 
                        min_lr=1e-7,
                        verbose=1)

model_check = ModelCheckpoint( #에포크마다 현재 가중치를 저장    
        filepath="./landmark_densenet201_1113_9907.h5", #모델 파일 경로
        monitor='val_loss',  # val_loss 가 좋아지지 않으면 모델 파일을 덮어쓰지 않음.
        save_best_only=True)

# print(#################################################################################)   

callbacks = [earlystop, learning_rate_reduction, model_check]

for layer in densenet201_load.layers:
    layer.trainable = True
 
# with strategy.scope(): 
with tf.device('/device:GPU:1'):  
    model = densenet201_load
    model.summary()

    model.compile(loss='categorical_crossentropy',
                  optimizer=Adam(learning_rate=0.000049),
                  metrics=['categorical_accuracy'])
    
    history = model.fit(dataset,
                        epochs=10,
                        validation_data=valid_dataset,
                        callbacks = callbacks)

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
densenet201 (Functional)     (None, 8, 15, 1920)       18321984  
_________________________________________________________________
global_average_pooling2d (Gl (None, 1920)              0         
_________________________________________________________________
dense (Dense)                (None, 1049)              2015129   
Total params: 20,337,113
Trainable params: 20,108,057
Non-trainable params: 229,056
_________________________________________________________________
Epoch 1/10
7049/7049 [==============================] - 5195s 737ms/step - loss: 0.0045 - categorical_accuracy: 0.9992 - val_loss: 0.0677 - val_categorical_accuracy: 0.9848
Epoch 2/10
7049/7049 [==============================] - 5188s 736ms/step - loss: 0.0040 - categorical_accuracy: 0.9994 - val_loss: 0.0475 - val_categorical_accuracy: 0.9885
Epoch 3/10
7049/7049 [